 SQL
===

SQL is a language for getting data from databases. It is also becoming essential to have some basic familiarity with SQL because it is universally used to slice and dice data across many different data storage formats and technologies.

We will only show the use of SQL as a query language here, since that is probably all that a statistician will use SQL for. However, SQL can also be used to create and modify tables, as well as manage database permissions.

Three ways of using SQL in Python will be shown:

- Using SQL to query `pandas` data frames
- Using SQL to query traditional database
- Using SQL via Jupyter `magic` functions

SQL via `pandas` DataFrames
----

We will use the [pandas-sql](https://github.com/yhat/pandasql/) package to practice SQL syntax for querying a `pandas` DataFrame. Later, we will see how to use an actual relational database.

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

In [2]:
from pandasql import PandaSQL
pdsql = PandaSQL()

In [3]:
tips = sns.load_dataset('tips')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


### Using SQL for Queries

Note that SQL is case-insensitive, but it is traditional to use ALL CAPS for SQL keywords. It is also standard to end SQL statements with a semi-colon.

### Simple Queries

### Specifiying columns to return

### Aliasing

### Filtering

### Filtering on strings

### Ordering

### Aggregate queries

Joins
----

A **join** occurs when you combine information from two or more database tables, based on information in a column that is common among the tables. As usual, it is easier to understand the concept with examples.

In [4]:
student = pd.read_csv('data/student.txt')
student

,student_id,first,last,email,major_id
0,1,frodo,baggins,frodo.baggins@duke.edu,1
1,2,bilbo,baggins,b_baggins@duke.edu,3
2,3,golum,golum,golum.golum@duke.edu,2
3,4,gandalf,white,g.white@duke.edu,5
4,5,gandalf,grey,g.grey@duke.edu,6
5,6,saruman,wise,s.wise@duke.edu,2


In [5]:
cls = pd.read_csv('data/class.txt')
cls

,class_id,code,name,credits
0,1,ANT01,Introduction to Hobbits,4
1,2,MAT802,Abstrct Nonsense,8
2,3,ENG234,Jabberwocky,2
3,4,STA007,Statistics for Secret Agens,4
4,5,PHY211,Physics of Star Wars,4


In [6]:
major = pd.read_csv('data/major.txt')
major

,major_id,name
0,1,Computer Science
1,2,Physics
2,3,Statisitcs
3,4,English
4,5,History


In [7]:
student_cls = pd.read_csv('data/student_class.txt')
student_cls

,student_id,class_id
0,1,3
1,1,4
2,2,1
3,2,4
4,3,1
5,3,2
6,3,3
7,3,5
8,4,2
9,4,5


### Matching students and majors

#### Inner join

#### Left outer join

SQL also has RIGHT OUTER JOIN and FULL OUTER JOIN but these are not currently supported by SQLite3 (the database engine used by `pdsql`).

#### Emulating a full outer join with UNION ALL

Only necessary if the database does not proivde FULL OUTER JOIN

#### Using linker tables to match students to classes (a MANY TO MANY join)

### Using SQLite3

[SQLite3](https://docs.python.org/3.5/library/sqlite3.html) is part of the standard library. However, the mechanics of using essentially any database in Python is similar, because of the Python [DB-API](https://www.python.org/dev/peps/pep-0249/).

In [8]:
import sqlite3

In [9]:
c = sqlite3.connect('data/Chinook_Sqlite.sqlite')

### SQLite specific commands to get metadata

Unlike SQL syntax for queries, how you get metadata from a relational database is vendor-specific. You'll have to read the docs to find out what is needed for your SQL flavor.

#### What tables are there in the database?

In [10]:
list(c.execute("SELECT name FROM sqlite_master WHERE type='table';"))

[('Album',),
 ('Artist',),
 ('Customer',),
 ('Employee',),
 ('Genre',),
 ('Invoice',),
 ('InvoiceLine',),
 ('MediaType',),
 ('Playlist',),
 ('PlaylistTrack',),
 ('Track',)]

#### What are the columns of the table "Album"?

In [11]:
list(c.execute("PRAGMA table_info(Album);"))

[(0, 'AlbumId', 'INTEGER', 1, None, 1),
 (1, 'Title', 'NVARCHAR(160)', 1, None, 0),
 (2, 'ArtistId', 'INTEGER', 1, None, 0)]

### Standard SQL statements with parameter substitution

Note: Using Python string substitution for Python defined parameters is dangerous because of the risk of [SQL injection attacks](http://xkcd.com/327/). Use parameter substitution with `?` instead. 

#### Do this

In [12]:
t = ['%rock%', 10]
list(c.execute("SELECT * FROM Album WHERE Title like ? AND ArtistID > ? LIMIT 5;", t))

[(59, 'Deep Purple In Rock', 58),
 (108, 'Rock In Rio [CD1]', 90),
 (109, 'Rock In Rio [CD2]', 90),
 (213,
  'Pure Cult: The Best Of The Cult (For Rockers, Ravers, Lovers & Sinners) [UK]',
  139),
 (216, 'Hot Rocks, 1964-1971 (Disc 1)', 142)]

#### Not this

In [13]:
t = ("'%rock%'", 10)
list(c.execute("SELECT * FROM Album WHERE Title like %s AND ArtistID > %d LIMIT 5;" % t))

[(59, 'Deep Purple In Rock', 58),
 (108, 'Rock In Rio [CD1]', 90),
 (109, 'Rock In Rio [CD2]', 90),
 (213,
  'Pure Cult: The Best Of The Cult (For Rockers, Ravers, Lovers & Sinners) [UK]',
  139),
 (216, 'Hot Rocks, 1964-1971 (Disc 1)', 142)]

User defined functions
----

Sometimes it is useful to have custom functions that run on the database server rather than on the client. These are called User Defined Functions or UDF. How do to do this varies with the database used, but it is fairly simple with Python and SQLite.

#### A standard UDF

In [14]:
def encode(text, offset):
    """Caesar cipher of text with given offset."""
    from string import ascii_lowercase, ascii_uppercase
    tbl = dict(zip(map(ord, ascii_lowercase + ascii_uppercase), 
               ascii_lowercase[offset:] + ascii_lowercase[:offset] + 
               ascii_uppercase[offset:] + ascii_uppercase[:offset]))
    return text.translate(tbl)

In [15]:
c.create_function("encode", 2, encode)

In [16]:
list(c.execute("SELECT Title, encode(Title, 2) FROM Album limit 5;"))

[('For Those About To Rock We Salute You',
  'Hqt Vjqug Cdqwv Vq Tqem Yg Ucnwvg Aqw'),
 ('Balls to the Wall', 'Dcnnu vq vjg Ycnn'),
 ('Restless and Wild', 'Tguvnguu cpf Yknf'),
 ('Let There Be Rock', 'Ngv Vjgtg Dg Tqem'),
 ('Big Ones', 'Dki Qpgu')]

#### An aggregate UDF

We can also add aggregate UDFs similar to SQL MIN, SUM, COUNT etc. Aggregate UDFs require you to write a class `__init__`, `step` and `finalize` methods.

In [17]:
class CV:
    """Aggregate UDF for coefficient of varation in %."""

    def __init__(self):
        self.s = []

    def step(self, value):
        self.s.append(value)

    def finalize(self):
        if len(self.s) < 2:
            return 0
        else:
            return 100.0*np.std(self.s)/np.mean(self.s)

In [18]:
c.create_aggregate("cv", 1, CV)

In [19]:
list(c.execute("PRAGMA table_info(Invoice);"))

[(0, 'InvoiceId', 'INTEGER', 1, None, 1),
 (1, 'CustomerId', 'INTEGER', 1, None, 0),
 (2, 'InvoiceDate', 'DATETIME', 1, None, 0),
 (3, 'BillingAddress', 'NVARCHAR(70)', 0, None, 0),
 (4, 'BillingCity', 'NVARCHAR(40)', 0, None, 0),
 (5, 'BillingState', 'NVARCHAR(40)', 0, None, 0),
 (6, 'BillingCountry', 'NVARCHAR(40)', 0, None, 0),
 (7, 'BillingPostalCode', 'NVARCHAR(10)', 0, None, 0),
 (8, 'Total', 'NUMERIC(10,2)', 1, None, 0)]

In [20]:
list(c.execute("SELECT cv(Total) from Invoice limit 10;"))

[(83.85715075292478,)]

In [21]:
c.close()

### Using SQL magic functions

We will use the [ipython-sql](https://github.com/catherinedevlin/ipython-sql) notebook extension for convenience. This will only work in notebooks and IPython scripts with the .ipy extension.

In [22]:
%load_ext sql

#### Configuring the SqlMagic extension

In [23]:
%config SqlMagic

SqlMagic options
--------------
SqlMagic.autolimit=<Int>
    Current: 0
    Automatically limit the size of the returned result sets
SqlMagic.autopandas=<Bool>
    Current: False
    Return Pandas DataFrames instead of regular result sets
SqlMagic.column_local_vars=<Bool>
    Current: False
    Return data into local variables from column names
SqlMagic.displaylimit=<Int>
    Current: 0
    Automatically limit the number of rows displayed (full result set is still
    stored)
SqlMagic.dsn_filename=<Unicode>
    Current: 'odbc.ini'
    Path to DSN file. When the first argument is of the form [section], a
    sqlalchemy connection string is formed from the matching section in the DSN
    file.
SqlMagic.feedback=<Bool>
    Current: True
    Print number of rows affected by DML
SqlMagic.short_errors=<Bool>
    Current: True
    Don't display the full traceback on SQL Programming Error
SqlMagic.style=<Unicode>
    Current: 'DEFAULT'
    Set the table printing style to any of prettytable's d

In [24]:
%config SqlMagic.displaylimit=10

#### Connect to SQLite3 database

In [25]:
%sql sqlite:///data/Chinook_Sqlite.sqlite

'Connected: None@data/Chinook_Sqlite.sqlite'

#### Other databases

See [SQLAlchemy connection strings](http://docs.sqlalchemy.org/en/latest/core/engines.html) for how to connect to other databases such as Oracle, MySQL or PostgreSQL.

#### Line magic

In [26]:
%sql SELECT * from Album LIMIT 5;

Done.


AlbumId,Title,ArtistId
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [27]:
%sql SELECT * from Artist LIMIT 5;

Done.


ArtistId,Name
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains


#### Cell magic

In [28]:
%%sql
SELECT Artist.Name, Album.Title
FROM Album
INNER JOIN Artist on Album.ArtistId = Artist.ArtistId
ORDER BY Artist.Name ASC
LIMIT 5;

Done.


Name,Title
AC/DC,For Those About To Rock We Salute You
AC/DC,Let There Be Rock
Aaron Copland & London Symphony Orchestra,"A Copland Celebration, Vol. I"
Aaron Goldberg,Worlds
Academy of St. Martin in the Fields & Sir Neville Marriner,The World of Classical Favourites


### You can assign results of queries to Python names

In [29]:
result = %sql SELECT * from Album;

Done.


In [30]:
type(result)

sql.run.ResultSet

#### Resutls behave like lists

In [31]:
result[2:4]

[(3, 'Restless and Wild', 2), (4, 'Let There Be Rock', 1)]

#### You can use Python variables in your queires.

Use `:varname` where you want to use a Python variable in your query. 

In [32]:
artist_id = 10

%sql select * from Artist where ArtistId < :artist_id;

Done.


ArtistId,Name
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains
6,Antônio Carlos Jobim
7,Apocalyptica
8,Audioslave
9,BackBeat


In [33]:
word = '%rock%'

%sql select * from Album WHERE Title LIKE :word;

Done.


AlbumId,Title,ArtistId
1,For Those About To Rock We Salute You,1
4,Let There Be Rock,1
59,Deep Purple In Rock,58
108,Rock In Rio [CD1],90
109,Rock In Rio [CD2],90
213,"Pure Cult: The Best Of The Cult (For Rockers, Ravers, Lovers & Sinners) [UK]",139
216,"Hot Rocks, 1964-1971 (Disc 1)",142


#### Convert to `pandas` dataframe

In [34]:
df = result.DataFrame()
df.head(5)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
